##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: 代理和 Barista Bot 的自動函式呼叫


這個筆記本展示了使用 Gemini API 的 Python SDK 使用自動函式呼叫建立代理的實用範例。你將定義整合到 Gemini API 的一些職能，建構咖啡廳的訂購系統，然後編寫一個代理程式迴圈，與使用者互動來訂購咖啡廳飲料。

這份指南的靈感來自於 ReAct 型式的 [Barista bot](https://aistudio.google.com/app/prompts/barista-bot) 提示，可透過 AI Studio 取得。


In [11]:
!pip install -qU google-generativeai

要執行此筆記本，你的 API 金鑰必須儲存在名為「GOOGLE_API_KEY」的 Colab 密碼中。如果你是在不同環境中執行，則可以將金鑰儲存在環境變數中。參閱 [驗證](../quickstarts/Authentication.zh.ipynb) 進一步瞭解。


In [12]:
from random import randint
from typing import Iterable

import google.generativeai as genai
from google.api_core import retry

from google.colab import userdata
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

## 定義 API

為了模擬咖啡廳的訂購系統，請定義以下功能來管理客戶訂單：增加、編輯、清除、確認和完成。

這些函式使用全域變數 `order` (進行中的訂單) 和 `placed_order` (已確認並傳送至廚房的訂單) 來追蹤客戶訂單。每個訂單編輯功能都會更新 `order`，且一旦提交，`order` 即會複製至 `placed_order` 並清除。

在 Python SDK 中，你可以直接將函式傳給模型建構函式，其中 SDK 會檢查類型簽章和文件字串以定義 `tools`。因此，正確設定每個參數的型別，並賦予函式合理的名稱和詳細的文件字串非常重要。


In [13]:
order = []  # The in-progress order.
placed_order = []  # The confirmed, completed order.

def add_to_order(drink: str, modifiers: Iterable[str] = ()) -> None:
  """Adds the specified drink to the customer's order, including any modifiers."""
  order.append((drink, modifiers))


def get_order() -> Iterable[tuple[str, Iterable[str]]]:
  """Returns the customer's order."""
  return order


def remove_item(n: int) -> str:
  """Remove the nth (one-based) item from the order.

  Returns:
    The item that was removed.
  """
  item, modifiers = order.pop(int(n) - 1)
  return item


def clear_order() -> None:
  """Removes all items from the customer's order."""
  order.clear()


def confirm_order() -> str:
  """Asks the customer if the order is correct.

  Returns:
    The user's free-text response.
  """

  print('Your order:')
  if not order:
    print('  (no items)')

  for drink, modifiers in order:
    print(f'  {drink}')
    if modifiers:
      print(f'   - {", ".join(modifiers)}')

  return input('Is this correct? ')


def place_order() -> int:
  """Submit the order to the kitchen.

  Returns:
    The estimated number of minutes until the order is ready.
  """
  placed_order[:] = order.copy()
  clear_order()

  # TODO(you!): Implement coffee fulfilment.
  return randint(1, 10)

## 測試 API

有了這些函式，測試它們是否如預期般運作。


In [14]:
# Test it out!

clear_order()
add_to_order('Latte', ['Extra shot'])
add_to_order('Tea')
remove_item(2)
add_to_order('Tea', ['Earl Grey', 'hot'])
confirm_order();

Your order:
  Latte
   - Extra shot
  Tea
   - Earl Grey, hot
Is this correct? yes it is!


## 定義提示

在此處定義完整的咖啡機器人提示。此提示包含咖啡廳的菜單項目和修改器以及一些說明。

說明包括關於如何呼叫功能的指南 (例如在呼叫 `place_order` 之前「務必使用 `confirm_order` 向使用者確認」)。你可以修改此內容以將自己的互動樣式新增至機器人，例如，如果你希望機器人在將物品新增至訂單之前重複每項要求，可以在此處提供說明。

提示結尾包括機器人可能遇到的某些術語和說明_du jour_ - 在此案例中，它指出咖啡廳已經用完豆漿。


In [15]:
COFFEE_BOT_PROMPT = """\You are a coffee order taking system and you are restricted to talk only about drinks on the MENU. Do not talk about anything but ordering MENU drinks for the customer, ever.
Your goal is to do place_order after understanding the menu items and any modifiers the customer wants.
Add items to the customer's order with add_to_order, remove specific items with remove_item, and reset the order with clear_order.
To see the contents of the order so far, call get_order (by default this is shown to you, not the user)
Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will display the order items to the user and returns their response to seeing the list. Their response may contain modifications.
Always verify and respond with drink and modifier names from the MENU before adding them to the order.
If you are unsure a drink or modifier matches those on the MENU, ask a question to clarify or redirect.
You only have the modifiers listed on the menu below: Milk options, espresso shots, caffeine, sweeteners, special requests.
Once the customer has finished ordering items, confirm_order and then place_order.

Hours: Tues, Wed, Thurs, 10am to 2pm
Prices: All drinks are free.

MENU:
Coffee Drinks:
Espresso
Americano
Cold Brew

Coffee Drinks with Milk:
Latte
Cappuccino
Cortado
Macchiato
Mocha
Flat White

Tea Drinks:
English Breakfast Tea
Green Tea
Earl Grey

Tea Drinks with Milk:
Chai Latte
Matcha Latte
London Fog

Other Drinks:
Steamer
Hot Chocolate

Modifiers:
Milk options: Whole, 2%, Oat, Almond, 2% Lactose Free; Default option: whole
Espresso shots: Single, Double, Triple, Quadruple; default: Double
Caffeine: Decaf, Regular; default: Regular
Hot-Iced: Hot, Iced; Default: Hot
Sweeteners (option to add one or more): vanilla sweetener, hazelnut sweetener, caramel sauce, chocolate sauce, sugar free vanilla sweetener
Special requests: any reasonable modification that does not involve items not on the menu, for example: 'extra hot', 'one pump', 'half caff', 'extra foam', etc.

"dirty" means add a shot of espresso to a drink that doesn't usually have it, like "Dirty Chai Latte".
"Regular milk" is the same as 'whole milk'.
"Sweetened" means add some regular sugar, not a sweetener.

We have run out of Soy milk today, so soy is not available.
"""

## 設定模型

在這個步驟中，你把函式合併成一個傳入 `tools` 的「系統」，實例化模型並開始聊天對話。

此區塊包含與 Gemini API 互動的兩個選項。透過切換 `use_sys_inst`，你可以切換使用帶有系統指令的 Gemini 1.5 Pro (品質最高，但免費額度可能會不足以進行長時間聊天對話) 或 Gemini 1.0 Pro (免費額度較高，但不支援系統指令)。

也可以定義一個可重新嘗試的 `set_message` 函式，以處理額度不足的對話。


In [17]:
ordering_system = [add_to_order, get_order, remove_item, clear_order, confirm_order, place_order]

# Toggle this to switch between Gemini 1.5 with a system instruction, or Gemini 1.0 Pro.
use_sys_inst = False

model_name = 'gemini-1.5-pro-latest' if use_sys_inst else 'gemini-1.0-pro-latest'

if use_sys_inst:
  model = genai.GenerativeModel(
      model_name, tools=ordering_system, system_instruction=COFFEE_BOT_PROMPT)
  convo = model.start_chat(enable_automatic_function_calling=True)

else:
  model = genai.GenerativeModel(model_name, tools=ordering_system)
  convo = model.start_chat(
      history=[
          {'role': 'user', 'parts': [COFFEE_BOT_PROMPT]},
          {'role': 'model', 'parts': ['OK I understand. I will do my best!']}
        ],
      enable_automatic_function_calling=True)


@retry.Retry(initial=30)
def send_message(message):
  return convo.send_message(message)


placed_order = []
order = []

## 與 Barista Bot 聊天

在模型定義且聊天建立後，剩下的就是將使用者輸入與模型做連結，並顯示輸出，透過迴圈的方式。此迴圈會持續執行直到下單。

在 Colab 執行時，任何固定寬度的文字來自你的 Python 程式碼 (例如訂購系統中的 `print` 呼叫)，一般文字來自 Gemini API，且輪廓框允許使用者的輸入，並以開頭的 `>` 呈現。

立即試用！


In [19]:
from IPython.display import display, Markdown

print('Welcome to Barista bot!\n\n')

while not placed_order:
  response = send_message(input('> '))
  display(Markdown(response.text))


print('\n\n')
print('[barista bot session over]')
print()
print('Your order:')
print(f'  {placed_order}\n')
print('- Thanks for using Barista Bot!')

Welcome to Barista bot!


> Tea, Earl Grey, Hot.


OK, I have added an Earl Grey tea, hot, to your order.

> and a strong latte please


OK, and I have added a latte with double espresso to your order.

> great, that's all thanks
Your order:
  Earl Grey
   - Hot
  Latte
   - Double
Is this correct? make it so!


Your order is confirmed, and it will be ready in about 4 minutes.




[barista bot session over]

Your order:
  [('Earl Grey', ['Hot']), ('Latte', ['Double'])]

- Thanks for using Barista Bot!


一些嘗試：
* 詢問菜單 (例如「有哪些咖啡飲品？」) 
* 使用未在提示中指定的術語 (例如「一杯濃郁拿鐵」或「一杯 EB 茶」) 
* 中途改變主意 (「呃取消拿鐵抱歉」) 
* 偏離菜單 (「一杯小調皮」）


## 另見

本範例應用程式展示如何整合傳統軟體系統 (咖啡訂購功能) 和由 Gemini API 支援的 AI 代理。這是一種使用 LLM 的簡單實用方式，允許使用開放且自然的人類語言輸入和輸出，但仍可讓人類參與，以確保正確操作。

若要深入瞭解 Barista Bot 的運作方式，請查看：

* [Barista Bot](https://aistudio.google.com/app/prompts/barista-bot) 提示
* [系統說明](../quickstarts/System_instructions.zh.ipynb)
* [自動函式呼叫](../quickstarts/Function_calling.zh.ipynb)
